In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

<b><font size="6" color="red"> ch11. 데이터프레임과 시리즈(Pandas)</font></b>
# 9절. 데이터 그룹화 및 집계

In [4]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 Groupby
cf. pivot_table

In [12]:
# 1) 그룹화 방법 1
#      df.groupby(by=[그롭핑할 열])[함수적용해서 출력할 열들].기초통계함수()
#             [함수적용해서 출력할 열들] 생략시 그룹핑할 열을 제외한 모든 열 적용
iris_grouped = iris.groupby(iris.Species)
iris_grouped = iris.groupby(['Species'])
iris_grouped = iris.groupby('Species')
for group, data in iris_grouped:
    print(group)
    display(data.sample(2).sort_index())

setosa


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
41,4.5,2.3,1.3,0.3,setosa
43,5.0,3.5,1.6,0.6,setosa


versicolor


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
74,6.4,2.9,4.3,1.3,versicolor
95,5.7,3.0,4.2,1.2,versicolor


virginica


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
106,4.9,2.5,4.5,1.7,virginica
107,7.3,2.9,6.3,1.8,virginica


In [7]:
iris.groupby(by='Species').mean()[['Petal.Length','Petal.Width']]
iris.groupby(by='Species')[['Petal.Length','Petal.Width']].mean()

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [20]:
iris_grouped_sum = iris.groupby('Species').median()
# iris_grouped_sum.reset_index()
iris_grouped_sum

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3
virginica,6.5,3.0,5.55,2.0


In [21]:
iris_grouped_sum.loc[['setosa','versicolor']]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3


In [17]:
iris_grouped_sum2 = iris.groupby('Species', as_index=False).sum()
iris_grouped_sum2

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [22]:
iris_grouped_sum2[(iris_grouped_sum2.Species=='setosa') |
                  (iris_grouped_sum2.Species=='versicolor')]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


In [ ]:
#      df.groupby(by=[그롭핑할 열])[함수적용해서 출력할 열들].함수()
# 2) 그룹화 방법 2
#      df.pivoy_table(index=[그룹핑할 열],
#                     values=[함수 적용해서 출력할 열들],
#                     aggfunc=함수(기본값 : mean))

In [23]:
iris.groupby('Species')['Petal.Length','Petal.Width'].mean()
iris.pivot_table(index='Species',
                 values=['Petal.Length','Petal.Width'],
                 aggfunc='mean')

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [33]:
# iris.groupby('Species').count() # 결측치를 제외한 값의 개수
iris.pivot_table(index='Species',
                 aggfunc='count')

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


### 다중열로 그룹화

In [34]:
# iris로 다중열로 그룹화 하기 위해 데이터 컬럼 추가(num 필드)
iris.shape

(150, 5)

In [36]:
import numpy as np
np.ravel([[i]*25 for i in range(6)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [39]:
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
75,6.6,3.0,4.4,1.4,versicolor,3
100,6.3,3.3,6.0,2.5,virginica,4
125,7.2,3.2,6.0,1.8,virginica,5


In [44]:
# 각 열의 임의의 데이터에 결측치를 할당
# iris.iloc[0,0] = np.nan
iris.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0


In [46]:
import random
print(random.sample(range(150), 3))
for col in range(iris.shape[1]-2): # iris.shape[1]-2 : 4
    iris.iloc[random.sample(range(150), col+1), col] = np.nan

[99, 74, 106]


In [47]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  149 non-null    float64
 1   Sepal.Width   148 non-null    float64
 2   Petal.Length  147 non-null    float64
 3   Petal.Width   146 non-null    float64
 4   Species       150 non-null    object 
 5   num           150 non-null    int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [49]:
iris.groupby(by=['Species', 'num'])[['Petal.Length','Petal.Width']].count()

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           24
           1              25           23
versicolor 2              25           25
           3              24           25
virginica  4              23           24
           5              25           25

In [50]:
iris.pivot_table(index=['Species','num'],
                 values = ['Petal.Length','Petal.Width'],
                 aggfunc='count')

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           24
           1              25           23
versicolor 2              25           25
           3              24           25
virginica  4              23           24
           5              25           25

In [51]:
iris.groupby('Species').describe()

Sepal.Length                                                 ...   num                                        
                  count      mean       std  min    25%  50%  75%  max  ... count mean       std  min  25%  50%  75%  max
Species                                                                 ...                                              
setosa             49.0  5.014286  0.351188  4.3  4.800  5.0  5.2  5.8  ...  50.0  0.5  0.505076  0.0  0.0  0.5  1.0  1.0
versicolor         50.0  5.936000  0.516171  4.9  5.600  5.9  6.3  7.0  ...  50.0  2.5  0.505076  2.0  2.0  2.5  3.0  3.0
virginica          50.0  6.588000  0.635880  4.9  6.225  6.5  6.9  7.9  ...  50.0  4.5  0.505076  4.0  4.0  4.5  5.0  5.0

[3 rows x 40 columns]

## 9.2 레이블 인코딩

In [52]:
iris.drop('num', axis=1, inplace=True)
iris.head(1)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa


In [55]:
# 레이블 인코딩을 지원하는 클래스 import
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # 객체생성
iris['target'] = le.fit_transform(iris.Species)
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,target
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
75,6.6,3.0,4.4,1.4,versicolor,1
100,6.3,3.3,6.0,2.5,virginica,2
125,7.2,3.2,6.0,1.8,virginica,2


# 10절. 데이터 구조 변경
- 와이드포맷 <-> 롱포맷
- 판다스 공식 문서의 melt :
https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt
- Tidy Data(교안 7p) : https://vita.had.co.nz/papers/tidy-data.pdf

In [57]:
from statsmodels.api import datasets
airquality = datasets.get_rdataset('airquality',
                                   package='datasets').data
airquality.head(2)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2


## 10.1 melt를 이용한 언피벗팅(와이드포맷 -> 롱포맷)

In [63]:
air_melted = airquality.melt(id_vars=['Month','Day'],
                            var_name='변수명', # 기본값 : variable
                            value_name='값')  # 기본값 : value
air_melted.iloc[::20]
air_melted.sort_values(by=['Month','Day'], inplace=True)
air_melted

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0
...,...,...,...,...
610,9,29,Temp,76.0
152,9,30,Ozone,20.0
305,9,30,Solar.R,223.0
458,9,30,Wind,11.5


In [65]:
# melt 연습(부동산 와이드포맷 데이터를 롱포맷으로 변환)
%ls C:\ai_x\download\shareData\부동산_250213\

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3664-591E

 C:\ai_x\download\shareData\부동산_250213 디렉터리

2025-04-30  오후 02:44    <DIR>          .
2025-04-30  오후 02:44    <DIR>          ..
2021-01-02  오후 03:44             2,163 전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv
2025-04-30  오후 02:04           419,340 주택도시보증공사_전국 신규 민간아파트 분양가격 동향_20241231.csv
               2개 파일             421,503 바이트
               2개 디렉터리  161,534,464,000 바이트 남음


In [68]:
import pandas as pd
df = pd.read_csv(r'C:\ai_x\download\shareData\부동산_250213\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv', 
                 encoding='cp949')
df

,지역,2013년12월,2014년1월,2014년2월,2014년3월,2014년4월,2014년5월,2014년6월,2014년7월,2014년8월,2014년9월,2014년10월,2014년11월,2014년12월,2015년1월,2015년2월,2015년3월,2015년4월,2015년5월,2015년6월,2015년7월,2015년8월
0,서울,18189,17925,17925,18016,18098,19446,18867,18742,19274,19404,19759,20242,20269,20670,20670,19415,18842,18367,18374,18152,18443
1,부산,8111,8111,9078,8965,9402,9501,9453,9457,9411,9258,9110,9208,9208,9204,9235,9279,9327,9345,9515,9559,9581
2,대구,8080,8080,8077,8101,8267,8274,8360,8360,8370,8449,8403,8439,8253,8327,8416,8441,8446,8568,8542,8542,8795
3,인천,10204,10204,10408,10408,10000,9844,10058,9974,9973,9973,10016,10020,10020,10017,9876,9876,9938,10551,10443,10443,10449
4,광주,6098,7326,7611,7346,7346,7523,7659,7612,7622,7802,7707,7752,7748,7752,7756,7861,7914,7877,7881,8089,8231
5,대전,8321,8321,8321,8341,8341,8341,8333,8333,8333,8048,8038,8067,8067,8067,8067,8067,8145,8272,8079,8079,8079
6,울산,8090,8090,8090,8153,8153,8153,8153,8153,8493,8493,8627,8891,8891,8526,8526,8629,9380,9192,9190,9190,9215
7,경기,10855,10855,10791,10784,10876,10646,10266,10124,10134,10501,10397,10356,10379,10391,10355,10469,10684,10685,10573,10518,10573
8,세종,7601,7600,7532,7814,7908,7934,8067,8067,8141,8282,8527,8592,8560,8560,8560,8555,8546,8546,8671,8669,8695
9,강원,6230,6230,6230,6141,6373,6350,6350,6268,6268,6419,6631,6365,6365,6348,6350,6182,6924,6846,6986,7019,7008


In [73]:
new_df = df.melt(id_vars='지역',
        var_name='연도월',
        value_name='평당분양가')
new_df.sample()

,지역,연도월,평당분양가
77,강원,2014년4월,6373


## 10.2 pivot_table을 이용한 피벗팅(롱포맷 -> 와이드포맷)

In [74]:
# air_melted를 와이드포맷으로
air_melted.head()

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0


In [78]:
airquality2 = air_melted.pivot_table(index=['Month','Day'],
                       columns='변수명',
                       values='값')
airquality2.reset_index(inplace=True)
airquality2.columns.name = None

In [79]:
airquality2.head()

,Month,Day,Ozone,Solar.R,Temp,Wind
0,5,1,41.0,190.0,67.0,7.4
1,5,2,36.0,118.0,72.0,8.0
2,5,3,12.0,149.0,74.0,12.6
3,5,4,18.0,313.0,62.0,11.5
4,5,5,NaN,NaN,56.0,14.3


# 11절. 데이터프레임(시리즈)에 함수 적용시키기
- apply(func, axis) : 축 요소 별(요소별) 함수 적용(데이터프레임/시리즈)
- applymap(func) : 각 요소별 함수 적용(데이터프레임)
- map(func) : 각 요소 별 함수 적용(1차원 리스트, 시리즈에서만 가능)

In [83]:
member = pd.read_csv('data/ch11_member.csv', comment='#')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [89]:
def get_year(datestr):
    return int(datestr.split('-')[0]) # year

In [91]:
get_year = lambda datestr : int(datestr.split('-')[0])

In [92]:
get_year('1999-05-01')

1999

In [93]:
get_month = lambda datestr : int(datestr.split('-')[1])

In [94]:
get_month('1999-05-01')

5

In [96]:
member['year'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[0]))
member.head()

,Name,Age,Email,Address,Birth,year
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999


In [99]:
X = iris.iloc[:, :-2]
# X의 열별 평균
avg = X.mean(axis=0)
avg

Sepal.Length    5.851678
Sepal.Width     3.050676
Petal.Length    3.714286
Petal.Width     1.215068
dtype: float64

In [100]:
X.iloc[0] -avg

Sepal.Length   -0.751678
Sepal.Width     0.449324
Petal.Length   -2.314286
Petal.Width    -1.015068
dtype: float64

In [102]:
# X의 모든 행에 대하여 열별 평균과의 거리를 수소점 2자리까지 출력
X.apply(lambda x : round(x-avg, 2), axis=1)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,-0.75,0.45,-2.31,-1.02
1,-0.95,-0.05,-2.31,-1.02
2,-1.15,0.15,-2.41,-1.02
3,-1.25,0.05,-2.21,-1.02
4,-0.85,0.55,-2.31,-1.02
...,...,...,...,...
145,0.85,-0.05,1.49,1.08
146,0.45,-0.55,1.29,0.68
147,0.65,-0.05,1.49,0.78
148,0.35,0.35,1.69,1.08


## 11.2 applymap

In [103]:
iris

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,target
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0
2,4.7,3.2,1.3,0.2,setosa,0
3,4.6,3.1,1.5,0.2,setosa,0
4,5.0,3.6,1.4,0.2,setosa,0
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,2
146,6.3,2.5,5.0,1.9,virginica,2
147,6.5,3.0,5.2,2.0,virginica,2
148,6.2,3.4,5.4,2.3,virginica,2


## 11.3 map
- 1차원

In [105]:
member['Month'] = member['Birth'].map(lambda datestr : int(datestr.split('-')[1]))
member

,Name,Age,Email,Address,Birth,year,Month
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999,1
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999,12
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999,11
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999,5
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999,1


In [125]:
# 연습하기(apply, map)
# new_df의 연도월 데이터를 연도와 열을 정수로 분리하여 column 추가
new_df['연도'] = new_df['연도월'].apply(lambda datestr : int(datestr.split('년')[0]))
# new_df['월'] = new_df['연도월'].apply(lambda datestr : int(datestr.split('년')[1].split('월')[0]))
new_df['월'] = new_df['연도월'].map(lambda datestr : int(datestr.split('년')[1][:-1]))
new_df

,지역,연도월,평당분양가,연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12
...,...,...,...,...,...
352,전북,2015년8월,6580,2015,8
353,전남,2015년8월,6289,2015,8
354,경북,2015년8월,7037,2015,8
355,경남,2015년8월,7665,2015,8


In [126]:
new_df.drop(['연도','월'],axis=1, inplace=True)
new_df.head()

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098


In [ ]:
# ★ 2차원 데이터 프레임의 apply 이용

In [128]:
new_df.head(2)

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111


In [138]:
def year_month_append(df) :
    'df를 받아 연도와 월 추가 후 return'
    df = df.copy() # 깊은복사(원본손상에 대한 경고 및 위험 방지) - 강의 중엔 덮어써서 사용
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1].replace('월', ''))
    return df

In [139]:
year_month_append(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
연도           2013
월              12
Name: 0, dtype: object

In [140]:
new_df = new_df.apply(year_month_append, axis=1)
new_df

,지역,연도월,평당분양가,연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12
...,...,...,...,...,...
352,전북,2015년8월,6580,2015,8
353,전남,2015년8월,6289,2015,8
354,경북,2015년8월,7037,2015,8
355,경남,2015년8월,7665,2015,8


# 12절. 일괄변경하기(결측치, 특정값)
## 12.1 fillna
- 대부분 결측치는 drop / 평균값대체 / 중위수대체 / 그룹별 중위수 / 이전값 / 이후값 등으로 대체

In [143]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3     , 4, np.nan, 1],
                   [np.nan, 3, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                  columns = [list('ABCD')])
df

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [144]:
# 모든 결측치를 0으로 대체
df.fillna(0)

,A,B,C,D
0,0.0,2,0.0,0
1,3.0,4,0.0,1
2,0.0,3,0.0,5
3,0.0,3,0.0,4


In [145]:
# 결측치를 이전행의 값으로 대체
df.fillna(method='ffill')

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [146]:
# 결측치를 다음행의 값으로 대체
df.fillna(method='bfill')

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [151]:
# 결측치를 열마다 특정값으로 대체
values = {'A':85, 'B':80, 'C':75, 'D':90}
df.fillna(value=values)

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,75.0,5
3,85.0,3,75.0,4


In [149]:
# 결측치를 열별 평균값으로 대체
df.mean(axis=0)
df.fillna(value=df.mean(axis=0))

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [150]:
# 결측치 대체를 2번째 결측치까지만 대체
print(values)
df.fillna(value=values, limit=2)

{'A': 85, 'B': 80, 'C': 75, 'D': 90}


,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,NaN,5
3,NaN,3,NaN,4


In [10]:
# (1단계) iris 각 열마다 20개씩 인위적으로 결측치를 할당한다
        # 결측치가 각 열마다 20개씩인지 확인
# (2-1단계) 결측치는 각 열의 평균으로 대체한다 : fillna 이용
# (2-2단계) 결측치는 종별 열별 평균(groupby, pivot_table)으로 대체한다 : fillna 사용 X, apply 사용
import random
import numpy as np
import pandas as pd
import seaborn as sns
iris = sns.load_dataset('iris')
rowlen, collen = iris.shape
rowlen, collen

(150, 5)

In [19]:
# 1단계 결측치 할당
for col in range(collen-1) :
    iris.iloc[random.sample(range(rowlen),20), col] = np.nan

In [30]:
# 결측치 확인 - iris.isna() : 결측치인지 여부 확인
# iris.isna().sum()
print(iris.isnull().sum())
display(iris.tail())

sepal_length    20
sepal_width     20
petal_length    20
petal_width     20
species          0
dtype: int64


,sepal_length,sepal_width,petal_length,petal_width,species
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


In [35]:
# 결측치가 하나라도 있는 행 출력
iris[iris['sepal_length'].isna() | iris['sepal_width'].isna() |
     iris['petal_length'].isna() | iris['petal_width'].isna()]

,sepal_length,sepal_width,petal_length,petal_width,species
0,NaN,3.5,1.4,0.2,setosa
4,5.0,NaN,1.4,0.2,setosa
6,NaN,3.4,1.4,0.3,setosa
9,NaN,3.1,1.5,0.1,setosa
10,5.4,3.7,NaN,0.2,setosa
...,...,...,...,...,...
136,6.3,3.4,NaN,2.4,virginica
139,6.9,3.1,5.4,NaN,virginica
140,6.7,3.1,NaN,NaN,virginica
142,NaN,2.7,5.1,1.9,virginica


In [36]:
# (2-1 단계) 결측치는 각 열의 평균으로 대체한다 : fillna 사용
# iris.median(numeric_only=True)
# iris.iloc[:,:-1].mean() # 기본값 : axis=0
iris.fillna(iris.iloc[:,:-1].mean())

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.853077,3.500000,1.4,0.2,setosa
1,4.900000,3.000000,1.4,0.2,setosa
2,4.700000,3.200000,1.3,0.2,setosa
3,4.600000,3.100000,1.5,0.2,setosa
4,5.000000,3.058462,1.4,0.2,setosa
...,...,...,...,...,...
145,6.700000,3.000000,5.2,2.3,virginica
146,6.300000,2.500000,5.0,1.9,virginica
147,6.500000,3.000000,5.2,2.0,virginica
148,6.200000,3.400000,5.4,2.3,virginica


In [37]:
iris.isna().sum()

sepal_length    20
sepal_width     20
petal_length    20
petal_width     20
species          0
dtype: int64

In [38]:
# (2-2단계) 결측치는 종별 열별 중위수(groupby, pivot_table)로 대체한다 : fillna 사용 x
# 종별 열별 중위수
iris_mean = iris.groupby('species').mean()
iris_mean

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,4.995349,3.421739,1.459574,0.240909
versicolor,5.944186,2.726190,4.276190,1.337209
virginica,6.602273,2.992857,5.548780,2.011628


In [43]:
# fillna 사용 X 확인
iris.iloc[2]

sepal_length       4.7
sepal_width        3.2
petal_length       1.3
petal_width        0.2
species         setosa
Name: 2, dtype: object

In [46]:
iris_mean #, iris_mean.loc['setosa', 'sepal_length']

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,4.995349,3.421739,1.459574,0.240909
versicolor,5.944186,2.726190,4.276190,1.337209
virginica,6.602273,2.992857,5.548780,2.011628


In [51]:
# iris 한 행을 매개변수로 받아 결측치가 있는 경우 종별 열별 중위수로 대체하는 함수
def iris_fillna(irisrow) :
    irisrow = irisrow.copy()
    for col, value in irisrow.items():
        if isinstance(value, (int, float, np.number)) and np.isnan(value) :
#             print(col, '이 결측치')
            irisrow[col] = iris_mean.loc[irisrow.species, col]
    return irisrow

In [52]:
iris_fillna(iris.iloc[2])

sepal_length       4.7
sepal_width        3.2
petal_length       1.3
petal_width        0.2
species         setosa
Name: 2, dtype: object

In [53]:
# apply 적용
iris.apply(iris_fillna, axis=1)

,sepal_length,sepal_width,petal_length,petal_width,species
0,4.995349,3.500000,1.4,0.2,setosa
1,4.900000,3.000000,1.4,0.2,setosa
2,4.700000,3.200000,1.3,0.2,setosa
3,4.600000,3.100000,1.5,0.2,setosa
4,5.000000,3.421739,1.4,0.2,setosa
...,...,...,...,...,...
145,6.700000,3.000000,5.2,2.3,virginica
146,6.300000,2.500000,5.0,1.9,virginica
147,6.500000,3.000000,5.2,2.0,virginica
148,6.200000,3.400000,5.4,2.3,virginica


# 12.2 replace(to_value, new_value, inplace=T/F)
- to_value를 new_value
- to_value에 정규표현식 이용
- 데이터프레임, 시리즈 적용

In [56]:
s = pd.Series([0,1,2,3,np.nan])
s.replace(np.nan, 0)
s.replace([0,1], 99) # 0이나 1을 99로 변경
s.replace([0,1], [100, 101]) # 0을 100으로, 1을 101로 변경

0    100.0
1    101.0
2      2.0
3      3.0
4      NaN
dtype: float64

In [59]:
df = pd.DataFrame({'A':['bat', 'foo','bait'],
                   'B':['abc', 'bar','xyz']})
df.replace(r'ba.+','XX', regex=True) # regex=True : 첫번째 인자를 정규표현식으로 인식

,A,B
0,XX,abc
1,foo,XX
2,XX,xyz


## 12.3 mask(조건), where(조건)
- where(조건) : 조건이 True인 요소만 출력하고, 그 외 요소는 결측치로 출력
- mask(조건) : 조건이 False인 요소만 출력하고, 그 외 요소는 결측치로 출력

In [60]:
s = pd.Series(range(10,20))
s

0    10
1    11
2    12
3    13
4    14
5    15
6    16
7    17
8    18
9    19
dtype: int64

In [61]:
s.where(s>15)

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6    16.0
7    17.0
8    18.0
9    19.0
dtype: float64

In [62]:
s.mask(s>15)

0    10.0
1    11.0
2    12.0
3    13.0
4    14.0
5    15.0
6     NaN
7     NaN
8     NaN
9     NaN
dtype: float64

## 12.4 dropna
- 결측치가 있는 데이터 누락(행,열)

In [63]:
df = pd.DataFrame({'name':['홍','김','이', np.nan],
                   'age': [np.nan, 20, 30, np.nan],
                   'born':['서울','부산',np.nan, np.nan]})
df

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,NaN
3,NaN,NaN,NaN


In [64]:
df.dropna(how='all') # 모두 결측치인 행만 제거 , axis=0 : 기본값

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,NaN


In [65]:
df.dropna() # 결측치가 하나라도 있는 행 삭제

,name,age,born
1,김,20.0,부산


In [66]:
df.dropna(axis=1) # 결측치가 하나라도 있는 열 삭제

""
0
1
2
3


In [68]:
df.dropna(subset=['name','age'])

,name,age,born
1,김,20.0,부산
2,이,30.0,NaN


# 13절. 시리즈
- 1차원 데이터

In [69]:
s = pd.Series([1000, 7000, 5000],
              index=['mango','apple','banana'])
s

mango     1000
apple     7000
banana    5000
dtype: int64

In [70]:
s['mango'], s.mango

(1000, 1000)

In [71]:
s['mango'] = 2800 # 데이터 수정
s['orange'] = 4000 # 데이터 삽입(추가)
s

mango     2800
apple     7000
banana    5000
orange    4000
dtype: int64

In [72]:
s['mango':'banana'] # 슬라이싱[from:to] - from부터 to까지 (to포함)

mango     2800
apple     7000
banana    5000
dtype: int64

In [73]:
s[['mango','banana']]

mango     2800
banana    5000
dtype: int64

In [74]:
s.drop('banana')

mango     2800
apple     7000
orange    4000
dtype: int64

In [75]:
s.sort_index()

apple     7000
banana    5000
mango     2800
orange    4000
dtype: int64

In [76]:
s.sort_values() # 기본 : 오름차순정렬

mango     2800
orange    4000
banana    5000
apple     7000
dtype: int64

In [77]:
s.sort_values(ascending=False)

apple     7000
banana    5000
orange    4000
mango     2800
dtype: int64

In [79]:
s.to_frame().T

,mango,apple,banana,orange
0,2800,7000,5000,4000


# 14절. 연습문제

In [82]:
import seaborn as sns
iris = sns.load_dataset("iris")
type(iris)

pandas.core.frame.DataFrame

In [83]:
# 1.iris 데이터에서 처음 다섯개 행만 출력하세요
iris.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [101]:
# 2.iris 데이터를 데이터프레임 변수인 독립변수 X와 종속변수 y로 나누세요. 종속변수는 species 열입니다.
X = iris.drop('species', axis=1)  # 독립변수: species를 제외한 나머지 열
y = iris['species']               # 종속변수: species 열
# iloc 이용
X = iris.iloc[:,:-1]
y = iris.iloc[:,-1]
# loc 이용
X = iris.loc[:, 'sepal_length':'petal_width']
y = iris.loc[:,'species':]
display(X)
y.to_frame()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


,species
0,setosa
1,setosa
2,setosa
3,setosa
4,setosa
...,...
145,virginica
146,virginica
147,virginica
148,virginica


In [89]:
# 3.iris 데이터에서 처음 50개행을 빼내서 temp변수에 저장하세요
temp = iris.iloc[:50]
temp

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


In [107]:
# 4.3번에서 선택한 데이터프레임의 요약정보를 출력하세요. 모든 열에 대해 요약정보가 출력되어야 합니다.
temp.describe(include=['float64','object'])

,sepal_length,sepal_width,petal_length,petal_width,species
count,50.00000,50.000000,50.000000,50.000000,50
unique,NaN,NaN,NaN,NaN,1
top,NaN,NaN,NaN,NaN,setosa
freq,NaN,NaN,NaN,NaN,50
mean,5.00600,3.428000,1.462000,0.246000,NaN
std,0.35249,0.379064,0.173664,0.105386,NaN
min,4.30000,2.300000,1.000000,0.100000,NaN
25%,4.80000,3.200000,1.400000,0.200000,NaN
50%,5.00000,3.400000,1.500000,0.200000,NaN
75%,5.20000,3.675000,1.575000,0.300000,NaN


In [91]:
# 5.versicolor종의 데이터만 iris_versicolor변수에 저장하세요
iris_versicolor = iris[iris['species'] == 'versicolor']
iris_versicolor

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
53,5.5,2.3,4.0,1.3,versicolor
54,6.5,2.8,4.6,1.5,versicolor
55,5.7,2.8,4.5,1.3,versicolor
56,6.3,3.3,4.7,1.6,versicolor
57,4.9,2.4,3.3,1.0,versicolor
58,6.6,2.9,4.6,1.3,versicolor
59,5.2,2.7,3.9,1.4,versicolor


In [92]:
# 6.2번의 X와 y변수를 합해서 iris_df데이터 프레임으로 만드세요
iris_df = pd.concat([X, y], axis=1)
iris_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [93]:
# 7. iris 데이터의 각 열 평균값을 출력하세요.
iris.describe().loc['mean']
iris.iloc[:,:-1].mean(axis=0)
iris.iloc[:,:-1].apply(np.mean, axis=0)

sepal_length    5.843333
sepal_width     3.057333
petal_length    3.758000
petal_width     1.199333
Name: mean, dtype: float64

In [94]:
# 8. iris 데이터의 각 열들 사이의 상관계수를 출력하세요.
iris.corr(numeric_only=True)

,sepal_length,sepal_width,petal_length,petal_width
sepal_length,1.000000,-0.117570,0.871754,0.817941
sepal_width,-0.117570,1.000000,-0.428440,-0.366126
petal_length,0.871754,-0.428440,1.000000,0.962865
petal_width,0.817941,-0.366126,0.962865,1.000000


In [108]:
# 9. iris 데이터에서 각 요소들과 변수별 평균값의 차이를 출력하세요. 다음 그림은 처음 다섯 개 행의 출력 결과입니다.
#    sepal_length  sepal_width  petal_length  petal_width
# 0     -0.743333      0.442667       -2.358     -0.999333
# 1     -0.943333     -0.057333       -2.358     -0.999333
# 2     -1.143333      0.142667       -2.458     -0.999333
# 3     -1.243333      0.042667       -2.258     -0.999333
# 4     -0.843333      0.542667       -2.358     -0.999333
iris_diff = iris.drop('species', axis=1) - iris.drop('species', axis=1).mean()
iris_diff.head()

,sepal_length,sepal_width,petal_length,petal_width
0,-0.743333,0.442667,-2.358,-0.999333
1,-0.943333,-0.057333,-2.358,-0.999333
2,-1.143333,0.142667,-2.458,-0.999333
3,-1.243333,0.042667,-2.258,-0.999333
4,-0.843333,0.542667,-2.358,-0.999333


In [96]:
# 10. iris 데이터의 종별 평균을 출력하세요.
iris.groupby('species').mean(numeric_only=True)

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [ ]:
# 11. iris 데이터에서 각 요소들과 종별 변수의 평균과의 차이를 출력하세요. 각 종별로 3개씩 출력하세요. 다음 그림은 출력 예입니다.
#    petal_length  petal_width  sepal_length  sepal_width species
# 0        -0.062       -0.046         0.094        0.072     NaN
# 1        -0.062       -0.046        -0.106       -0.428     NaN
# 2        -0.162       -0.046        -0.306       -0.228     NaN
# 50        0.440        0.074         1.064        0.430     NaN
# 51        0.240        0.174         0.464        0.430     NaN
# 52        0.640        0.174         0.964        0.330     NaN
# 100       0.448        0.474        -0.288        0.326     NaN
# 101      -0.452       -0.126        -0.788       -0.274     NaN
# 102       0.348        0.074         0.512        0.026     NaN

In [109]:
iris_mean_by_species = iris.groupby(iris.species).mean()
iris_mean_by_species

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [110]:
iris.loc[0] - iris_mean_by_species.loc[iris.loc[0].species]

petal_length   -0.062
petal_width    -0.046
sepal_length    0.094
sepal_width     0.072
species           NaN
dtype: object

In [ ]:
pd.options.display.max_rows = 9

In [113]:
result = iris.apply(lambda series : series-iris_mean_by_species.loc[series.species], axis=1)
# result.iloc[list(range(3))+list(range(50,53))+list(range(100,103))]
result.iloc[[0,1,2,50,51,52,100,101,102]]
result.groupby(iris.species).head(3)

,petal_length,petal_width,sepal_length,sepal_width,species
0,-0.062,-0.046,0.094,0.072,NaN
1,-0.062,-0.046,-0.106,-0.428,NaN
2,-0.162,-0.046,-0.306,-0.228,NaN
50,0.440,0.074,1.064,0.430,NaN
51,0.240,0.174,0.464,0.430,NaN
52,0.640,0.174,0.964,0.330,NaN
100,0.448,0.474,-0.288,0.326,NaN
101,-0.452,-0.126,-0.788,-0.274,NaN
102,0.348,0.074,0.512,0.026,NaN


In [112]:
result.groupby(iris.species).head(3)

,petal_length,petal_width,sepal_length,sepal_width,species
0,-0.062,-0.046,0.094,0.072,NaN
1,-0.062,-0.046,-0.106,-0.428,NaN
2,-0.162,-0.046,-0.306,-0.228,NaN
50,0.440,0.074,1.064,0.430,NaN
51,0.240,0.174,0.464,0.430,NaN
52,0.640,0.174,0.964,0.330,NaN
100,0.448,0.474,-0.288,0.326,NaN
101,-0.452,-0.126,-0.788,-0.274,NaN
102,0.348,0.074,0.512,0.026,NaN


In [115]:
# 11 각 요소들과 종별 변수의 평균과의 차이를 각 종별로 3개씩 출력(방법2)
iris_grouped_sample = iris.groupby('species').head(3)
iris_grouped_sample

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
100,6.3,3.3,6.0,2.5,virginica
101,5.8,2.7,5.1,1.9,virginica
102,7.1,3.0,5.9,2.1,virginica


In [116]:
iris_mean_by_species

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [118]:
temp = iris_grouped_sample.iloc[0]
temp - iris_mean_by_species.loc[temp.species]

petal_length   -0.062
petal_width    -0.046
sepal_length    0.094
sepal_width     0.072
species           NaN
dtype: object

In [119]:
iris_grouped_sample.apply(lambda temp:temp - iris_mean_by_species.loc[temp.species], 
                         axis=1)

,petal_length,petal_width,sepal_length,sepal_width,species
0,-0.062,-0.046,0.094,0.072,NaN
1,-0.062,-0.046,-0.106,-0.428,NaN
2,-0.162,-0.046,-0.306,-0.228,NaN
50,0.440,0.074,1.064,0.430,NaN
51,0.240,0.174,0.464,0.430,NaN
52,0.640,0.174,0.964,0.330,NaN
100,0.448,0.474,-0.288,0.326,NaN
101,-0.452,-0.126,-0.788,-0.274,NaN
102,0.348,0.074,0.512,0.026,NaN
